# Day 9 - Rope Bridge
## Data

In [1]:
example_data = """
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
""".strip()

print(example_data)

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2


In [2]:
import aocd
raw_data = aocd.get_data(year=2022, day=9)
print(raw_data)

L 1
D 2
R 2
L 1
D 1
L 1
U 1
R 1
L 2
R 2
L 2
D 1
R 2
D 1
U 2
R 2
D 1
R 1
L 2
R 1
D 1
U 2
R 2
D 1
R 2
L 1
D 1
U 1
R 1
D 2
L 1
D 1
L 1
U 1
L 2
U 1
L 1
U 1
L 1
D 2
R 2
U 1
D 2
R 1
U 1
D 1
R 1
U 2
L 2
D 2
R 1
U 2
L 2
U 1
D 1
L 1
R 2
L 2
R 1
D 2
L 1
D 2
L 1
R 2
U 2
D 2
U 1
R 2
D 2
L 2
U 1
D 2
R 1
L 2
R 1
L 2
U 2
D 2
U 2
D 2
R 1
U 2
L 2
D 1
U 2
L 1
D 1
R 2
U 1
L 1
D 1
U 2
D 2
R 2
U 1
L 2
D 2
L 1
D 2
L 1
U 1
R 2
L 2
D 1
R 2
U 2
R 2
L 2
D 1
R 1
L 1
D 2
U 2
L 1
D 1
R 1
L 3
U 2
R 2
D 3
L 1
R 3
L 1
R 2
D 1
U 3
L 1
U 2
L 1
D 3
R 3
D 1
U 3
R 2
D 2
R 2
L 1
U 1
R 1
U 2
R 3
U 1
D 1
U 1
R 3
D 2
R 2
U 3
D 1
R 3
L 3
U 3
D 3
R 1
D 3
L 1
D 3
L 1
R 1
U 1
L 2
D 3
U 3
D 2
U 3
D 2
U 2
D 1
R 2
U 3
L 1
R 2
L 3
U 3
D 3
L 2
R 2
U 3
R 1
L 1
U 1
D 1
R 2
L 3
U 3
D 1
L 1
U 1
R 1
D 2
L 2
D 2
R 3
D 2
R 3
D 2
R 1
D 1
R 3
D 1
U 1
L 1
R 2
D 3
L 3
R 1
L 2
R 1
U 3
D 2
R 1
U 1
L 2
R 2
L 2
D 1
U 3
R 3
D 2
R 1
U 2
L 3
R 1
D 1
U 2
L 4
D 2
R 1
U 4
R 4
D 3
R 3
U 2
L 2
U 1
D 3
U 2
D 2
R 4
D 3
R 3
D 4
R 2
U 2
L 1
D 1
L 2
U 4
D 4
R 3


## Parsing

In [62]:
import math

def cap(val, min, max):
    if val < min:
        return min
    elif val > max:
        return max
    else:
        return val
    

class Vector(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    @property
    def length(self):
        # Returns the length of the vecotr
        return math.sqrt(self.x**2 + self.y**2)
    
    def normalize(self):
        # Returns a new vector where x and y are in the set {-1, 0, 1}
        x = cap(self.x, -1, 1)
        y = cap(self.y, -1, 1)

        return Vector(x, y)
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    
    
assert Vector(2, 3) + (Vector(1, 1) * 5) == Vector(7, 8)

assert Vector(3, 2) - Vector(2, 3) == Vector(1, -1)

assert Vector(3, 4).length == 5.0

assert Vector(3, 4).normalize() == Vector(1, 1)

assert Vector(-2, 0).normalize() == Vector(-1, 0)


In [29]:
directions = {
    'U': Vector(0, 1),
    'D': Vector(0, -1),
    'L': Vector(-1, 0),
    'R': Vector(1, 0),
}


def parse_line(line):
    """Parse a line and return a vector: (x, y)."""
    tokens = line.split(' ')
    
    direction = tokens[0]
    unit_vector = directions[direction]
    
    distance = int(tokens[1])
    
    return [unit_vector] * distance


def parse(data):
    """Parse the data into a list of vectors: (x, y)."""
    
    lines = data.split('\n')
    return [parse_line(line) for line in lines]
    
    
example_movements = parse(example_data)
print(example_movements)

real_movements = parse(raw_data)

[[(1, 0), (1, 0), (1, 0), (1, 0)], [(0, 1), (0, 1), (0, 1), (0, 1)], [(-1, 0), (-1, 0), (-1, 0)], [(0, -1)], [(1, 0), (1, 0), (1, 0), (1, 0)], [(0, -1)], [(-1, 0), (-1, 0), (-1, 0), (-1, 0), (-1, 0)], [(1, 0), (1, 0)]]


## Part 1

In [80]:
class RopeSimulatey(object):
    def __init__(self, length = 1):
        self.head = Vector(0, 0)
        
        self.knots = [Vector(0, 0) for i in range (length)]
        self.tail_positions = {self.knots[-1]}
    
    def move(self, vector):
        """
        Move the head by the given vector, then if the distance between the head and tail >= 2, move the tail
        closer to the head.
        """
        self.head += vector
        next_knot = self.head
        
        for i, knot in enumerate(self.knots):
            difference = next_knot - knot
            distance = difference.length
            next_knot = knot

            if distance >= 2:
                new_position = knot + difference.normalize()
                # print(f"Moving knot {i+1} from {knot} to {new_position}")
                self.knots[i] = new_position
                
        self.tail_positions.add(self.knots[-1])
            
    def __repr__(self):
        return f"Head: {self.head}, Knots: {self.knots}"
    
    def simulate(self, movements):
        """Move the simulated rope through a series of movements."""
        
        for movement in movements:
            for move in movement:
                self.move(move)
        

test_simulatey = RopeSimulatey()
print(test_simulatey)

test_simulatey.simulate(example_movements)
print(test_simulatey)
    

Head: (0, 0), Knots: [(0, 0)]
Head: (2, 2), Knots: [(1, 2)]


In [81]:
def count_tail_positions(movements, length = 1):
    """
    Given a list of movements (direction, distance), return the count of all unique
    positions that the tail of the rope has occupied.
    """
    simulatey = RopeSimulatey(length)
    simulatey.simulate(movements)
    return len(simulatey.tail_positions)

assert count_tail_positions(example_movements) == 13
count_tail_positions(real_movements)

5619

## Part 2

In [86]:
assert count_tail_positions(example_movements, 9) == 1
count_tail_positions(real_movements, 9)

2376